# Core Concepts of Elasticsearch Query Toolkit

This notebook explores the fundamental concepts and components of the Elasticsearch Query Toolkit. Understanding these core concepts will provide a solid foundation for building and customizing Elasticsearch queries using the toolkit.

## The Directive Pattern

The Elasticsearch Query Toolkit is built around the **directive pattern**, which provides a declarative way to define query components. A directive is a self-contained unit that knows how to generate a specific part of an Elasticsearch query.

### Key Characteristics of Directives

- **Self-contained**: Each directive encapsulates a specific query operation
- **Composable**: Directives can be combined to create complex queries
- **Configurable**: Directives can be configured with fields, values, and parameters
- **Reusable**: The same directive can be used in multiple contexts

The directive pattern allows you to build queries in a modular, maintainable way, separating the query logic from the business logic of your application.

## Directive Types

The toolkit provides several types of directives, each serving a specific purpose:

### Match Directives

Match directives are used to match documents based on field values. The toolkit includes several types of match directives:

- **ConstMatchDirective**: Matches exact values (terms query)
- **TextMatchDirective**: Performs full-text search with options like fuzziness
- **RangeMatchDirective**: Matches values within a range
- **FieldExistsDirective**: Checks if a field exists
- **ScriptMatchDirective**: Uses a script to match documents
- **QueryStringMatchDirective**: Uses Elasticsearch's query string syntax
- **WaterfallFieldMatchDirective**: Tries multiple fields in sequence

### Boolean Directives

Boolean directives combine other directives using boolean logic:

- **AndDirective**: Requires all child directives to match
- **OrDirective**: Requires at least one child directive to match

### Custom Directives

- **CustomMatchDirective**: Allows you to create reusable, complex query patterns

### Score Function Directives

Score function directives are used with the FunctionScoreEngine to influence document scoring:

- **ScriptScoreDirective**: Uses a script to calculate scores
- **WeightDirective**: Applies a simple weight multiplier
- **DecayFunctionDirective**: Applies distance-based scoring (linear, exp, gauss)

## Directive Configuration

All directives share a common configuration pattern. Let's look at the key configuration methods:

In [ ]:
import json
from elastictoolkit.queryutils.builder.matchdirective import ConstMatchDirective
from elastictoolkit.queryutils.consts import FieldMatchType

# Create a directive
directive = ConstMatchDirective(rule=FieldMatchType.ANY, name="my_directive")

# Configure fields
directive.set_field("title", "description")

# Configure values
directive.set_values("elasticsearch", "python")

# Configure match parameters
directive.set_match_params({"param1": "value1", "param2": "value2"})

print(json.dumps(directive.to_dsl().to_query(), indent=2))

### Common Configuration Methods

- __set_field(*fields)__: Specifies the fields to match against
- __set_values(*values_list, **values_map)__: Sets the values to match
- __set_match_params(params)__: Sets runtime parameters for dynamic value resolution
- __set_match_query_extra_args(**kwargs)__: Sets additional query-specific parameters

### Field Match Types

The `rule` parameter in match directives determines how multiple values are combined:

- **FieldMatchType.ANY**: Match if any value matches (OR logic)
- **FieldMatchType.ALL**: Match only if all values match (AND logic)

### Match Modes

The `mode` parameter determines how the directive is applied:

- **MatchMode.INCLUDE**: Include documents that match (default)
- **MatchMode.EXCLUDE**: Exclude documents that match
- **MatchMode.INCLUDE_IF_EXIST_ANY**: Include documents only if any of the specified fields exist

## Engines

Engines orchestrate multiple directives to build complete queries. The toolkit provides two main types of engines:

### DirectiveEngine

The `DirectiveEngine` combines multiple match directives into a boolean query. It's the most common engine type and is used for standard search queries.

### FunctionScoreEngine

The `FunctionScoreEngine` extends from the same `BaseQueryEngine` as `DirectiveEngine` to support function score queries, which allow you to modify document scores based on factors other than relevance.

## Creating an Engine

Engines are created by subclassing `DirectiveEngine` or `FunctionScoreEngine` and defining directives as class attributes:

In [ ]:
from elastictoolkit.queryutils.builder.directiveengine import DirectiveEngine
from elastictoolkit.queryutils.builder.matchdirective import ConstMatchDirective, TextMatchDirective
from elastictoolkit.queryutils.consts import FieldMatchType

class SearchEngine(DirectiveEngine):
    # Define directives as class attributes
    category = ConstMatchDirective(rule=FieldMatchType.ANY)
    text_search = TextMatchDirective(rule=FieldMatchType.ANY)
    
    class Config:
        # Engine configuration
        value_mapper = None  # Will be set later

engine = SearchEngine()

## Value Mapping

Value mapping is a powerful feature that allows you to define field-value relationships in a centralized way. This is done using the `DirectiveValueMapper` class:

In [ ]:
from elastictoolkit.queryutils.builder.directivevaluemapper import DirectiveValueMapper
from elastictoolkit.queryutils.types import FieldValue

class SearchValueMapper(DirectiveValueMapper):
    # Map directive names to fields and values
    category = FieldValue(
        fields=["category", "subcategory"],
        values_list=["match_params.category"]
    )
    
    text_search = FieldValue(
        fields=["title", "description"],
        values_list=["match_params.search_text"]
    )

### Dynamic Value Resolution

Value mappers support dynamic value resolution using special syntax:

- **"match_params.key"**: Resolves to the value of `match_params["key"]`
- __"*match_params.key"__: Resolves to the list of values in `match_params["key"]` (if it's an iterable; eg: `list`)

This allows you to create flexible queries that adapt to runtime parameters.

## Query Building Workflow

Let's understand the workflow of how a query is built using the toolkit:

1. **Engine Initialization**: An engine instance is created with match parameters
2. **Directive Discovery**: The engine discovers directives defined as class attributes
3. **Value Mapping**: The engine uses the value mapper to resolve fields and values for each directive
4. **Directive Building**: Each directive is built with the resolved fields, values, and match parameters
5. **Query Generation**: Each directive generates its part of the query
6. **Query Composition**: The engine combines the individual query parts into a complete query

This workflow happens automatically when you call the `to_dsl()` method on an engine instance.

## Complete Example

Let's put it all together with a complete example:

In [ ]:
import json
from elastictoolkit.queryutils.builder.directiveengine import DirectiveEngine
from elastictoolkit.queryutils.builder.matchdirective import ConstMatchDirective, TextMatchDirective
from elastictoolkit.queryutils.builder.directivevaluemapper import DirectiveValueMapper
from elastictoolkit.queryutils.types import FieldValue
from elastictoolkit.queryutils.consts import FieldMatchType

# Define a value mapper
class ProductValueMapper(DirectiveValueMapper):
    category = FieldValue(
        fields=["category"],
        values_list=["match_params.category"]
    )
    search = FieldValue(
        fields=["name", "description"],
        values_list=["match_params.search_text"]
    )

# Define an engine
class ProductSearchEngine(DirectiveEngine):
    category = ConstMatchDirective(rule=FieldMatchType.ANY)
    search = TextMatchDirective(rule=FieldMatchType.ANY)
    
    class Config:
        value_mapper = ProductValueMapper()

# Create an engine instance with match parameters
engine = ProductSearchEngine().set_match_params(
    {
        "category": "electronics",
        "search_text": "wireless headphones"
    }
)

# Generate the query
query = engine.to_dsl()

# Print the query as JSON
print(json.dumps(query.to_query(), indent=2))

## Nested Fields

The toolkit supports Elasticsearch's nested fields through the `NestedField` class:

In [ ]:
from elastictoolkit.queryutils.types import NestedField
from elastictoolkit.queryutils.builder.matchdirective import ConstMatchDirective
from elastictoolkit.queryutils.consts import FieldMatchType

# Create a directive with nested fields
directive = ConstMatchDirective(rule=FieldMatchType.ANY)
directive.set_field(
    "regular_field",
    NestedField(nested_path="reviews", field_name="reviews.rating")
)
directive.set_values(5)
directive.set_match_params({})

# Generate the query
query = directive.to_dsl()
print(json.dumps(query.to_query(), indent=2))

## Error Handling

The toolkit provides clear error messages when directives are misconfigured. Common errors include:

- Missing required fields
- Missing required values
- Missing match parameters
- Invalid value types
- Using a CustomMatchDirective with an incompatible engine

Always check for these errors when debugging your queries.

## Next Steps

Now that you understand the core concepts of the Elasticsearch Query Toolkit, you can proceed to the **Basic Queries** notebook to learn more about working with individual directives in detail.